In [1]:
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools

Requirement already up-to-date: pyswarm in /usr/local/lib/python3.6/dist-packages (0.6)
Requirement already up-to-date: pactools in /usr/local/lib/python3.6/dist-packages (0.3)


In [2]:
from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils
from keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models

#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend 
from keras.utils import np_utils
from keras.utils import to_categorical

from numpy import savetxt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Running on PyMC3 v3.7


In [3]:
def data1():
  train1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train.npy',allow_pickle=True)
  train_labels1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train_labels.npy',allow_pickle=True)
  train2,test1, train_labels2,test_labels1 = train_test_split(train1, train_labels1, test_size=0.2,random_state=42)
  x_train=train2/225.0
  y_train = pd.get_dummies(train_labels2)

  x_test=test1/225.0
  y_test = pd.get_dummies(test_labels1)
  return x_train,y_train,x_test,y_test

In [4]:
x_train, y_train, x_test, y_test = data1()

In [5]:
IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')

In [6]:
#fine_tuning,flatvsglobalpool,dense_layers,dropout1,dropout2
lb=[0,0,0,0,0,0.001]
ub=[150,1,3,0.65,0.65,0.2]

In [7]:
def create_model_dense_resnet(x):
  print(x[0],x[1],x[2],x[3],x[4],x[5])
  IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

  resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet') 
  tempre=resnet
  for layer in tempre.layers[:(-1)*int(round(x[0]))]:
    layer.trainable = False
  
  #vgg19.trainable=False
  model = tf.keras.Sequential()
  model.add(tempre)
  if (int(round(x[1]))):
    model.add(keras.layers.Flatten())
  else:
    model.add(keras.layers.GlobalAveragePooling2D())
  
  if (int(round(x[2]))==3):
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(x[3]))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(x[4]))
  elif (int(round(x[2]))==2):
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(x[3]))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(x[4]))
  elif (int(round(x[2]))==1):
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(x[3]))
  else:
    pass

  for layer in model.layers:
    print(layer, layer.trainable)
  model.add(keras.layers.Dense(3,activation="softmax"))
  if x[5]< 0.003:
    learning_rate = 0.001
  elif x[5]< 0.0075:
    learning_rate = 0.005
  elif x[5]< 0.015:
    learning_rate = 0.01
  elif x[5]< 0.035:
    learning_rate = 0.02
  elif x[5]< 0.075:
    learning_rate = 0.05
  elif x[5]< 0.125:
    learning_rate = 0.1
  elif x[5]< 0.175:
    learning_rate = 0.15
  else:
    learning_rate = 0.2

  opt = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
  return model

In [8]:
EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')
count=0

In [9]:
def apple(x):
  model = create_model_dense_resnet(x)
  model.fit(x_train, y_train, epochs=20, batch_size=1000, verbose=1,validation_data=(x_test, y_test),callbacks=[EarlyStopper])
  loss, acc = model.evaluate(x_test, y_test, verbose=1)
  if acc>0.9:
    global count 
    count = count+1
    model.save(f"/content/drive/My Drive/saved_models/pso_resnet/model-{count}-{round(acc, 3)}-{round(loss, 3)}")
    savetxt(f"/content/drive/My Drive/saved_models/pso_resnet/data-{count}.csv", x, delimiter=',')
  return loss 

In [ ]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)
print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))

107.82377783015923 0.9518994896625311 2.363082911344248 0.38435019490964106 0.004659221447470646 0.06464708946708093
<tensorflow.python.keras.engine.functional.Functional object at 0x7fab877a0a20> True
<tensorflow.python.keras.layers.core.Flatten object at 0x7fab876e4240> True
<tensorflow.python.keras.layers.core.Dense object at 0x7fab876e4390> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7fab871509b0> True
<tensorflow.python.keras.layers.core.Dense object at 0x7fab87150c18> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7fab8715e898> True
Epoch 1/20
17/17 [==============================] - 12s 706ms/step - loss: 0.5745 - accuracy: 0.7709 - val_loss: 3.2528 - val_accuracy: 0.5585
Epoch 2/20
17/17 [==============================] - 9s 519ms/step - loss: 0.2758 - accuracy: 0.9026 - val_loss: 7.4075 - val_accuracy: 0.5558
Epoch 3/20
17/17 [==============================] - 9s 521ms/step - loss: 0.1833 - accuracy: 0.9363 - val_loss: 2.2702 - val_accuracy: 0.